# Data Aquisition

Ziel dieses Notebooks ist es einen Datensatz mit Spielerstatistiken zu generieren, welcher Spieler in `professional` und `nicht-professional` teilt. Dazu wird in einem ersten Schritt eine Liste mit FACEIT User-IDs von professionellen Spielern erfasst und darauf im zweiten Schritt Spielerstatistiken zu den Spielen dieser Spieler abgerufen. Der resultierende Datensatz, sowie einige Zwischenschritte, sind ebenfalls in diesem Repository gespeichert. Somit können einzelne Code-Snippets und auch folgende Notebooks ausgeführt werden, ohne ständig die Queries in diesem Notebook wiederholt auszuführen. Das spart zum einen Zeit (ca. 6h) als auch Rechenressourcen auf der eigenen Maschine und auf den FACEIT-Servern. Weiterhin ist zum jetzigen Zeitpunkt unklar ob die FACEIT-API Limits aufweißt und den API-Key ungültig macht.

In [2]:
import json
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd

import requests
from dotenv import load_dotenv
from tqdm import tqdm
import plotly.express as px
 

Laden des API-Keys. Dieser kann im Webportal der [FACEIT Data API](https://developers.faceit.com/docs/tools/data-api) erstellt werden.

In [2]:
load_dotenv()
headers = {"Authorization": f"Bearer {os.environ['FACEIT_API_KEY']}"}

## Liste mit professionellen Spielern

FACEIT stellt sogenannte `Hubs` zur Verfügung welche eine Ansammlung an Spielern darstellen. Der `FPL-CSGO-Europe` Hub ist die Offizielle Gruppierung aller Spieler die in den letzten Jahren an professionellen Turnieren teilgenommen haben. Somit können die Mitglieder dieses Hubs als Grundwahrheit ob ein Spieler professionell ist angesehen werden. Über die [FACEIT Data API](https://developers.faceit.com/docs/tools/data-api) werden die User-IDs (und weitere, für dieses Projekt irrelevante Features) abgerufen. Der Hub besteht aus insgesamt 281 Spielern.

Da FACEIT nicht erlaubt alle Spieler auf einmal abzufragen sondern lediglich 100 pro Anfrage werden drei Anfragen gestellt. Die abgefragten Daten werden daraufhin in einem Pandas DataFrame dargestellt und im `Feather` Dateiformat gespeichert. _Das Feather Dateiformat ist deutlich performanter als gewöhnliche CSV Dateien und ist speziell auf Dataframes angepasst. Jedoch ist es nicht 'human-readable', was in diesem Kontext aber nicht notwendig ist. Mehr zum Feather Dateiformat kann in [diesem Blogeintrag](https://towardsdatascience.com/stop-using-csvs-for-storage-this-file-format-is-150-times-faster-158bd322074e) nachgelesen werden._

In [13]:
FPL_CSGO_EUROPE_ID = "74caad23-077b-4ef3-8b1d-c6a2254dfa75"
hub_members = []
# Do three requests to API since max 'limit' is 100 and there are just under 300 players in the hub
for i in range(3):
    # offset, since limit is always 100
    start = i * 100
    r = requests.get(
        f'https://open.faceit.com/data/v4/hubs/{FPL_CSGO_EUROPE_ID}/members',
        params={'limit': 100, 'offset': start},
        headers=headers)
    hub_members += r.json()["items"]
# Create dataframe from queried data
professionals_df = pd.DataFrame(hub_members)
# Save dataframe in feather format
professionals_df.to_feather("../data/1a-professionals_df.feather")
professionals_df

,user_id,nickname,avatar,roles,faceit_url
0,7dbb06e0-a052-43ed-a4cb-24337980932c,FACEIT_Mikey,https://assets.faceit-cdn.net/avatars/7dbb06e0...,"[owner, member]",https://www.faceit.com/{lang}/players/FACEIT_M...
1,18911949-a23d-4b42-bd0b-7a970b51824c,FROZENN-_-,https://assets.faceit-cdn.net/avatars/18911949...,"[member, 3dfa861d-e495-4fce-b509-dc3ae30bfea9,...",https://www.faceit.com/{lang}/players/FROZENN-_-
2,1fd5bdf4-d03f-44fd-8141-a34fb890b602,innocent,,"[member, 50add7b6-c304-4fe4-bd02-7d4b6b82a369]",https://www.faceit.com/{lang}/players/innocent
3,fa59bd46-eaf7-4f75-a16a-7047ad64e224,NlKO,https://assets.faceit-cdn.net/avatars/fa59bd46...,"[member, 981770db-26ff-4d4b-a307-4626ca5986bc]",https://www.faceit.com/{lang}/players/NlKO
4,158f23b1-1e62-43a8-a24a-a20821a6e756,k1to0,https://assets.faceit-cdn.net/avatars/158f23b1...,"[member, 50add7b6-c304-4fe4-bd02-7d4b6b82a369]",https://www.faceit.com/{lang}/players/k1to0
...,...,...,...,...,...
276,fe480d93-45a1-437e-a8c8-a33a837524ef,flamie,https://distribution.faceit-cdn.net/images/250...,"[member, ff99e8ac-788e-413d-84e2-e7e89fb07993]",https://www.faceit.com/{lang}/players/flamie
277,bb11aade-ab7b-4b39-95b9-586bddfb0e12,Tuurtlezik,https://assets.faceit-cdn.net/avatars/bb11aade...,"[member, ce5661a4-45f5-4662-a60e-73e4151a4406]",https://www.faceit.com/{lang}/players/Tuurtlezik
278,798a2505-8b8f-45c0-a5b3-1d3e0952112a,J0TAAA-,https://assets.faceit-cdn.net/avatars/798a2505...,"[member, ce5661a4-45f5-4662-a60e-73e4151a4406]",https://www.faceit.com/{lang}/players/J0TAAA-
279,c0a979a3-7128-4a06-ba7a-8d6f7e8fbd2f,brnz4n,https://assets.faceit-cdn.net/avatars/c0a979a3...,"[member, ce5661a4-45f5-4662-a60e-73e4151a4406]",https://www.faceit.com/{lang}/players/brnz4n


## Query von Spiel-Statistiken

Die Statistiken aller `Matches` von einer Reihe an professionellen Spielern soll über die [FACEIT Data API](https://developers.faceit.com/docs/tools/data-api) erfasst werden. Dazu werden alle Spiele des letzten Jahres von den obigen professionellen Spielern abgefragt, maximal 100 Spiele pro Spieler um API-Limits zu vermeiden.

Die Informationen zu allen Spielen aus dem letzten Jahr von den übrigen 99 spielen werden abgerufen.

In [13]:
today = datetime.today()
last_year = today - relativedelta(years=1)
res = []
for player_id in tqdm(professionals_df["user_id"], "Getting player match-ids"):
    params = {
        'game': 'csgo',
        'from': last_year.timestamp(),
        'to': today.timestamp(),
        'limit': 100
    }
    r = requests.get(f'https://open.faceit.com/data/v4/players/{player_id}/history', headers=headers, params=params)
    res.append(r.json())
# Save results without any further processing in temporary json file <- this file is ignored from git
json.dump(res, open("../data/response-professional-player-match_infos.json", "w"), sort_keys=True)
res[0]

Getting player match-ids: 100%|██████████| 280/280 [09:16<00:00,  1.99s/it]


[{'items': [{'match_id': '1-83e8c84d-0719-4424-a79a-b238b7a3e1e6',
    'game_id': 'csgo',
    'region': 'EU',
    'match_type': '',
    'game_mode': '5v5',
    'max_players': 10,
    'teams_size': 5,
    'teams': {'faction2': {'team_id': '4a611867-e49b-463e-a6a9-a51969313590',
      'nickname': 'team_bodie',
      'avatar': 'https://assets.faceit-cdn.net/avatars/4c85d5c5-6e44-488e-9277-3d2540076358_1551110369856.jpg',
      'type': '',
      'players': [{'player_id': '4c85d5c5-6e44-488e-9277-3d2540076358',
        'nickname': 'bodie',
        'avatar': 'https://assets.faceit-cdn.net/avatars/4c85d5c5-6e44-488e-9277-3d2540076358_1551110369856.jpg',
        'skill_level': 9,
        'game_player_id': '76561198027833675',
        'game_player_name': 'BODIE',
        'faceit_url': 'https://www.faceit.com/{lang}/players/bodie'},
       {'player_id': '7dbb06e0-a052-43ed-a4cb-24337980932c',
        'nickname': 'FACEIT_Mikey',
        'avatar': 'https://assets.faceit-cdn.net/avatars/7dbb06e0-a0

In [6]:
# Load match infos from temporary json file
match_infos = json.load(open("../data/response-professional-player-match_infos.json"))
# Filter out errors and invalid requests
match_infos = [res_info['items'] for res_info in match_infos if 'items' in res_info.keys()]
# Flatten nested requests and extract all 'match_id's
match_ids = [match_info['match_id'] for res_items in match_infos for match_info in res_items]
len(match_ids)

26883

Es wurden 26883 Spiele gefunden. Zu jedem Spiel werden die Statistiken der einzelnen Spieler in diesem Spiel abgerufen. Da sehr viele (26883) Anfragen gestellt werden wird sicherheitshalber alle 1000 Anfragen der Zwischenstand gespeichert und kann bei Bedarf, sollte es z.B. durch ein Timeout oder Verbindungsverlust zu einem Fehler in der Abfrage kommen, von dem letztem Zwischenstand neu gestartet werden.

In [7]:
res = []

# Load temporary save if there is one
try:
    res = json.load(open("../data/response-professional-player-match_stats.json"))
    start_from = len(res)
except FileNotFoundError:
    start_from = 0

for i, match_id in enumerate(tqdm(match_ids[start_from:], "Getting match stats")):
    r = requests.get(f'https://open.faceit.com/data/v4/matches/{match_id}/stats', headers=headers)
    res.append(r.json())
    # Make a temporary save every 1000 requests
    if i % 1000 == 0:
        json.dump(res, open("../data/response-professional-player-match_stats.json", "w"), sort_keys=True)

json.dump(res, open("../data/response-professional-player-match_stats.json", "w"), sort_keys=True)
res[0]

Getting match stats: 0it [00:00, ?it/s]


{'rounds': [{'best_of': '2',
   'competition_id': None,
   'game_id': 'csgo',
   'game_mode': '5v5',
   'match_id': '1-83e8c84d-0719-4424-a79a-b238b7a3e1e6',
   'match_round': '1',
   'played': '1',
   'round_stats': {'Map': 'de_dust2',
    'Region': 'EU',
    'Rounds': '10',
    'Score': '9 / 1',
    'Winner': '8285f70d-324c-4f53-9585-55c6d5c01b4e'},
   'teams': [{'players': [{'nickname': 'ForceHunterX',
       'player_id': '46e04839-959b-4d96-a117-a28444470186',
       'player_stats': {'Assists': '0',
        'Deaths': '2',
        'Headshots': '10',
        'Headshots %': '62',
        'K/D Ratio': '8',
        'K/R Ratio': '1.6',
        'Kills': '16',
        'MVPs': '4',
        'Penta Kills': '0',
        'Quadro Kills': '1',
        'Result': '1',
        'Triple Kills': '0'}},
      {'nickname': 'R_SaitamaTV',
       'player_id': '8285f70d-324c-4f53-9585-55c6d5c01b4e',
       'player_stats': {'Assists': '2',
        'Deaths': '4',
        'Headshots': '1',
        'Headshots %

## JSON-Abfragen in DataFrames konvertieren

> Dieser Absatz kann auch ohne die vorherigen Teile ausgeführt werden (bis auf die Imports)

Zuerst wird der DataFrame mit der Liste an professionellen Spielern sowie die JSON-Datei mit allen 26883 Spiel-Statistiken geladen.

In [13]:
professionals_df = pd.read_feather("../data/1a-professionals_df.feather")
match_stats = json.load(open("../data/response-professional-player-match_stats.json"))
len(match_stats)

26883

FACEIT unterscheidet bei Spielen zwischen `match` und `round`. Eine Runde (`round`) ist gewonnen wenn ein Team entweder 16 Punkte erreicht hat, bei Overtime mehr als 2 Punkte abstand zum Gegner hat oder das andere Team aufgibt. Eine Partie (`match`) kann jedoch auch aus mehreren Runden bestehen. So wird in Turnieren häufig ein `BestOf3` gespielt, also bis ein Team zwei Runden gewonnen hat. `match_stats` ist eine Liste aus Dicts mit einem einzigen key: `rounds` also Runden. Wie viele Runden kann ein Match haben? Wie viele Runden wurden in dem Datensatz gespielt? Wie viele Matches konnten nicht von der FACEIT-API abgerufen werden?

In [14]:
num_rounds_min = np.inf
num_rounds_max = 0
num_rounds_sum = 0
unfound_matches = []
for stat in match_stats:
    try:
        n = len(stat['rounds'])
    except KeyError:
        unfound_matches.append(stat)
        continue
    if n > num_rounds_max:
        num_rounds_max = n
    if n < num_rounds_min:
        num_rounds_min = n
    num_rounds_sum += n
num_rounds_mean = num_rounds_sum / len(match_stats)
print(f"Wie viele Runden kann ein Match haben? {num_rounds_max}\nWie viele Runden wurden in dem Datensatz gespielt? {num_rounds_sum}\nWie viele Matches konnten nicht von der FACEIT-API abgerufen werden? {len(unfound_matches)}")

Wie viele Runden kann ein Match haben? 5
Wie viele Runden wurden in dem Datensatz gespielt? 28088
Wie viele Matches konnten nicht von der FACEIT-API abgerufen werden? 502


Die 502 ungefundenen Matches werden im folgenden vom Datensatz entfert und ignoriert. Was für Attribute besitzt eine Runde und wie sehen die Daten aus? (`teams` Attribut wurde entfert, da es eine komplexere Datenstruktur ist, dazu mehr weiter unten)

In [15]:
# Flatten match_stats into round_stats and ignore the unfound matches
round_stats = [round_stat for stat in match_stats if 'rounds' in stat.keys() for round_stat in stat['rounds']]
# What attributes has a single round?
round_keys = {}
for stat in round_stats:
    for k, v in stat.items():
        if k not in round_keys and k != 'teams':
            round_keys[k]= v
round_keys

{'best_of': '2',
 'competition_id': None,
 'game_id': 'csgo',
 'game_mode': '5v5',
 'match_id': '1-83e8c84d-0719-4424-a79a-b238b7a3e1e6',
 'match_round': '1',
 'played': '1',
 'round_stats': {'Map': 'de_dust2',
  'Region': 'EU',
  'Rounds': '10',
  'Score': '9 / 1',
  'Winner': '8285f70d-324c-4f53-9585-55c6d5c01b4e'}}

Anscheinend gibt es noch andere Spielmodie als der gewöhnliche 5-gegen-5 Spielmodus. Diese sind jedoch nicht relevant für den Datensatz. Entfernen aller nicht 5v5-Runden:

In [16]:
round_stats = [stat for stat in round_stats if stat['game_mode'] == "5v5"]
len(round_stats)

27400

Nach entfernen der Nicht-5v5-Runden bleiben 27400 Runden. Aus diesen können nun Features zu jedem Spieler extrahiert werden. Was für Attribute besitzt das `teams` Attribut einer Runde? (Auch hier wird ein Attribut, diesmal `players`, versteckt, da es eine weitere komplexere Datenstruktur darstellt)

In [17]:
team_keys = {}
for round_stat in round_stats:
    # Since teams is actually a list containing infos for both teams iterate over it
    for team in round_stat['teams']:
        for k, v in team.items():
            if k not in team_keys and k != 'players':
                team_keys[k] = v
team_keys
        

{'premade': False,
 'team_id': '8285f70d-324c-4f53-9585-55c6d5c01b4e',
 'team_stats': {'Final Score': '9',
  'First Half Score': '7',
  'Overtime score': '0',
  'Second Half Score': '2',
  'Team': 'team_R_SaitamaTV',
  'Team Headshots': '5',
  'Team Win': '1'}}

Welche Attribute besitzt das `players` Attribut eines Teams?

In [18]:
player_keys = {}
for round_stat in round_stats:
    # Since players is actually a list containing infos for all players iterate over it
    for team in round_stat['teams']:
        for player in team['players']:
            for k, v in player.items():
                if k not in player_keys:
                    player_keys[k] = v
player_keys

{'nickname': 'ForceHunterX',
 'player_id': '46e04839-959b-4d96-a117-a28444470186',
 'player_stats': {'Assists': '0',
  'Deaths': '2',
  'Headshots': '10',
  'Headshots %': '62',
  'K/D Ratio': '8',
  'K/R Ratio': '1.6',
  'Kills': '16',
  'MVPs': '4',
  'Penta Kills': '0',
  'Quadro Kills': '1',
  'Result': '1',
  'Triple Kills': '0'}}

Schlussendlich sieht die Datenstruktur der `round_stats` Liste wie folgt aus:

```python
round_stats = [
  {
    'best_of': '2',
    'competition_id': None,
    'game_id': 'csgo',
    'game_mode': '5v5',
    'match_id': '1-83e8c84d-0719-4424-a79a-b238b7a3e1e6',
    'match_round': '1',
    'played': '1',
    'round_stats': {
      'Map': 'de_dust2',
      'Region': 'EU',
      'Rounds': '10',
      'Score': '9 / 1',
      'Winner': '8285f70d-324c-4f53-9585-55c6d5c01b4e'
    },
    'teams': [
      {
        'premade': False,
        'team_id': '8285f70d-324c-4f53-9585-55c6d5c01b4e',
        'team_stats': {
          'Final Score': '9',
          'First Half Score': '7',
          'Overtime score': '0',
          'Second Half Score': '2',
          'Team': 'team_R_SaitamaTV',
          'Team Headshots': '5',
          'Team Win': '1'
        },
        'players': [
          {
            'nickname': 'ForceHunterX',
            'player_id': '46e04839-959b-4d96-a117-a28444470186',
            'player_stats': {
              'Assists': '0',
              'Deaths': '2',
              'Headshots': '10',
              'Headshots %': '62',
              'K/D Ratio': '8',
              'K/R Ratio': '1.6',
              'Kills': '16',
              'MVPs': '4',
              'Penta Kills': '0',
              'Quadro Kills': '1',
              'Result': '1',
              'Triple Kills': '0'
            }
          } # single player
        ] # players
      } # single team
    ] # teams
  } # single round_stat
] # round_stats
```

Es handelt sich somit um 3 Objekt-Typen welche in Listen ineinander gespeichert sind. Somit müssen über drei Schleifen die Daten-Features erfasst werden. Zu jedem Spieler jedes Teams jeder Runde aller Matches gibt es somit schlussendlich eine Zeile mit folgenden Daten-Features:

Aus dem Round-Objekt:

- Map
- Region
- Runden (Hiermit gemeint die Counter-Strike internen Runden, nicht die FACEIT-Runden)
  
Aus dem Team-Objekt:

- Gewonnen?
- Premade (Ob die Spieler sich schon vor dem Spiel kannten und entsprechend besser kommunizieren können)
- Score (Insgesammt, Erste Halbzeit, Zweite Halbzeit, Verlängerung)

Aus dem Spieler-Objekt:

- Nickname
- Player ID
- Ist Professionell?
- Assists
- Deaths
- Headshots
- Headshot Ratio
- K/D Ratio
- K/R Ratio
- Kills
- MVPs
- Ace
- Quad Kills
- Triple Kills

In [47]:
player_stats = []
for round_stat in tqdm(round_stats):
    # Used for determine the winner team in the next loop
    winner_team_id = round_stat['round_stats']['Winner']
    # Extract round specific features
    round_features = {
        "Map": round_stat['round_stats']['Map'],
        "Region": round_stat['round_stats']['Region'],
        "Rounds": int(round_stat['round_stats']['Rounds'])
    }
    for team in round_stat['teams']:
        # Extract team specific features
        team_features = {
            "Winner": winner_team_id == team['team_id'],
            "Premade": team['premade'],
            "Score": int(team['team_stats']['Final Score']),
            "Score First Half": int(team['team_stats']['First Half Score']),
            "Score Second Half": int(team['team_stats']['Second Half Score']),
            "Score Overtime": int(team['team_stats']['Overtime score'])
        }
        for player in team['players']:
            # Extract player specific features
            player_features = {
                "Nickname": player['nickname'],
                "Player ID": player['player_id'],
                "Professional": player['player_id'] in professionals_df["user_id"].values,
                "Assists": int(player['player_stats']['Assists']),
                "Deaths": int(player['player_stats']['Deaths']),
                "Headshots": int(player['player_stats']['Headshots']),
                "Headshot Ratio": float(player['player_stats']['Headshots %']) / 100,
                "K/D Ratio": float(player['player_stats']['K/D Ratio']),
                "K/R Ratio": float(player['player_stats']['K/R Ratio']),
                "Kills": int(player['player_stats']['Kills']),
                "MVPs": int(player['player_stats']['MVPs']),
                "Ace": int(player['player_stats']['Penta Kills']),
                "Quad Kills": int(player['player_stats']['Quadro Kills']),
                "Triple Kills": int(player['player_stats']['Triple Kills'])
            }

            # Combine Round-, Team- and Player-Features to one single dictionary and append this to the player_stats list
            player_stats.append(dict(
                **round_features,
                **team_features,
                **player_features
            ))
df = pd.DataFrame(player_stats)
df.to_feather("../data/1b-player_match_statistics.feather")
df

100%|██████████| 27400/27400 [00:09<00:00, 3001.12it/s]


,Map,Region,Rounds,Winner,Premade,Score,Score First Half,Score Second Half,Score Overtime,Nickname,...,Deaths,Headshots,Headshot Ratio,K/D Ratio,K/R Ratio,Kills,MVPs,Ace,Quad Kills,Triple Kills
0,de_dust2,EU,10,True,False,9,7,2,0,ForceHunterX,...,2,10,0.62,8.00,1.60,16,4,0,1,0
1,de_dust2,EU,10,True,False,9,7,2,0,R_SaitamaTV,...,4,1,0.20,1.25,0.50,5,1,0,0,1
2,de_dust2,EU,10,True,False,9,7,2,0,kubinatorLSD,...,2,4,0.40,5.00,1.00,10,3,0,0,1
3,de_dust2,EU,10,True,False,9,7,2,0,_mhN,...,5,6,0.75,1.60,0.80,8,1,0,0,0
4,de_dust2,EU,10,True,False,9,7,2,0,RUcKeTa,...,3,4,0.40,3.33,1.00,10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272214,de_mirage,SA,19,True,False,16,12,4,0,max_-,...,10,7,0.37,1.90,1.00,19,6,0,0,3
272215,de_mirage,SA,19,True,False,16,12,4,0,brnz4n,...,9,13,0.57,2.56,1.21,23,3,0,0,2
272216,de_mirage,SA,19,True,False,16,12,4,0,allen-_-,...,7,6,0.67,1.29,0.47,9,2,0,0,1
272217,de_mirage,SA,19,True,False,16,12,4,0,Timothyo,...,11,6,0.25,2.18,1.26,24,5,0,1,2


Schlussendlich konnten 272219 Datensätze erfasst werden. Jeder Datensatz weißt 23 Features auf.

## Alternative verarbeitung des Datensatzes

Im Folgenden werden zusätzlich die Statistiken aller Runden der Spieler kommuliert um zu jedem Spieler einzelne Datenpunkte zu erhalten.  

In [48]:
from dataclasses import dataclass, asdict

@dataclass()
class Player:
    user_id: str
    nickname: str
    professional: bool
    assists: float
    deaths: float
    headshots: float
    kills: float
    mvps: float
    aces: float
    quadkills: float
    triplekills: float
    entries: int = 1

    def add(self, assists: int, deaths: int, headshots: int, kills: int, mvps: int, aces: int, quadkills: int, triplekills: int):
        # Overall pattern for updating the ratios with new values:
        # updated_ratio = (ratio * entries + new_value) / (entries + 1)
        self.assists = (self.assists*self.entries + assists) / (self.entries + 1)
        self.deaths = (self.deaths*self.entries + deaths) / (self.entries + 1)
        self.headshots = (self.headshots*self.entries + headshots) / (self.entries + 1)
        self.kills = (self.kills*self.entries + kills) / (self.entries + 1)
        self.mvps = (self.mvps*self.entries + mvps) / (self.entries + 1)
        self.aces = (self.aces*self.entries + aces) / (self.entries + 1)
        self.quadkills = (self.quadkills*self.entries + quadkills) / (self.entries + 1)
        self.triplekills = (self.triplekills*self.entries + triplekills) / (self.entries + 1)
        self.entries += 1

df = pd.read_feather("../data/1b-player_match_statistics.feather")
player_stats = {}
for i, row in tqdm(df.iterrows(), "Calculating Player Stats", total=len(df)):
    row_stats = row[["Assists", "Deaths", "Headshots", "Kills", "MVPs", "Ace", "Quad Kills", "Triple Kills"]]
    try:
        player: Player = player_stats[row["Player ID"]]
        player.add(*row_stats)
    except:
        player_stats[row["Player ID"]] = Player(row["Player ID"], row["Nickname"], row["Professional"], *row_stats)

player_stats_list = []
for player in tqdm(player_stats.values(), "Converting to dict"):
    player_stats_list.append(asdict(player))
df = pd.DataFrame(player_stats_list)
df

Converting to dict: 100%|██████████| 39832/39832 [00:00<00:00, 57199.31it/s]


,user_id,nickname,professional,assists,deaths,headshots,kills,mvps,aces,quadkills,triplekills,entries
0,46e04839-959b-4d96-a117-a28444470186,ForceHunterX,False,0.0,2.0,10.0,16.0,4.0,0.0,1.0,0.0,3
1,8285f70d-324c-4f53-9585-55c6d5c01b4e,R_SaitamaTV,False,2.0,4.0,1.0,5.0,1.0,0.0,0.0,1.0,3
2,84a0d7de-4d9a-4807-afa7-f97108b6a857,kubinatorLSD,False,2.0,2.0,4.0,10.0,3.0,0.0,0.0,1.0,3
3,e2d99c1a-9d4f-47f6-86b0-12a05f5b12cf,_mhN,False,1.0,5.0,6.0,8.0,1.0,0.0,0.0,0.0,3
4,8c8e5ec5-4d48-4d3f-97c6-c0da31e8cabe,RUcKeTa,False,5.0,3.0,4.0,10.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
39827,11591d3a-8604-41d4-b019-961afa2ba5a9,Niqim,False,0.0,16.0,1.0,12.0,1.0,0.0,0.0,1.0,1
39828,41ca1ae7-47fd-4b28-bdf0-5c6e1a8f2ba4,chriiS-A,False,1.0,17.0,5.0,11.0,0.0,0.0,0.0,0.0,1
39829,7410dc5a-8299-4f1c-a857-733a9084aec9,natol,False,4.0,18.0,3.0,5.0,1.0,0.0,0.0,0.0,1
39830,3ebd1d0f-1e00-4357-ad9e-12da50805f3c,HitOne,False,3.0,16.0,4.0,14.0,1.0,0.0,0.0,2.0,1


Sehr viele Spieler besitzen nur wenige Einträge (`entries`) im Datensatz, da nur über die Liste der professionellen Spieler abgefragt wurde und somit das Auftreten von anderen Spielern eher zufällig ist. Spieler mit wenigen Einträgen sollen aus dem Datensatz entfernt werden, da diese wenig aussagekräftig sind. Wie viele Spieler haben mehr als `i ∈ [0,20]` Einträge?

In [49]:
y = [len(df[df["entries"] > i]) for i in range(0, 20)]
fig = px.line(y=y, title='Entries cap')
fig.show()

Es zeigt sich, dass mit 2966 Spielern relativ wenige Spieler mehr als 10 Einträge haben. Diese `i=10` wird im Folgenden willkürlich als Mindestanzahl an Einträgen eines Spielers genutzt. Alle Spieler mit weniger Einträgen werden aus dem Datensatz entfernt.

In [50]:
df = df[df["entries"] >= 10]
df = df.reset_index(drop=True)
df.to_feather("../data/1c-player_statistics.feather")
df

,user_id,nickname,professional,assists,deaths,headshots,kills,mvps,aces,quadkills,triplekills,entries
0,4c85d5c5-6e44-488e-9277-3d2540076358,bodie,True,2.765306,17.214286,5.612245,16.653061,2.132653,0.000000,0.234694,0.816327,98
1,7dbb06e0-a052-43ed-a4cb-24337980932c,FACEIT_Mikey,True,2.205128,17.871795,4.230769,10.461538,1.461538,0.000000,0.000000,0.666667,39
2,4b9f7374-18d7-4c4e-835b-06e5b0c7534b,TheOmicron,True,3.292035,18.654867,5.132743,16.469027,1.973451,0.017699,0.115044,0.734513,113
3,dd127ad1-08d9-413d-85b5-fa8f128e7ab9,Marty_-,False,3.097561,17.780488,7.024390,17.658537,2.804878,0.073171,0.097561,1.219512,41
4,be8c5c77-d299-47db-a0c7-3d42ac334c12,B0YSKI,False,3.028571,18.514286,4.657143,15.000000,1.628571,0.000000,0.142857,0.771429,35
...,...,...,...,...,...,...,...,...,...,...,...,...
2961,bfdd336a-a7e9-48d4-99be-2deb4775010e,BnTeT,False,4.000000,17.625000,9.750000,21.812500,3.125000,0.062500,0.437500,1.375000,16
2962,4ea5a5f4-7832-45a9-bc00-dbb36c145c7b,Mysil,False,3.586207,21.034483,7.241379,14.137931,1.793103,0.000000,0.137931,0.379310,29
2963,94517e34-08de-4e23-99f3-906d287ac035,jolietoG,False,3.590909,20.454545,10.000000,19.363636,2.818182,0.000000,0.363636,1.000000,22
2964,eb3b0e87-a327-4822-91a6-0751b7538d33,RiddareN-,False,3.000000,21.500000,9.833333,16.583333,2.250000,0.000000,0.166667,0.916667,12
